In [2]:
%pip install itables tqdm


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import math
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path
from tqdm import tqdm
from scipy.stats import entropy
from itertools import permutations


def safe_log2(x):
    return math.log2(x) if x > 0 else 0


def tuple_to_string(num_tuple):
    return ''.join(map(str, num_tuple))


def calculate_bulls_cows(source, target):
    if len(source) != len(target):
        raise ValueError("Input arrays must have the same length")

    bulls = sum(s == t for s, t in zip(source, target))
    common_digits = set(source) & set(target)
    cows = sum(min(source.count(digit), target.count(digit)) for digit in common_digits) - bulls

    return bulls, cows


def parse_bulls_n_cows_map_name(digits, guesses={}):
	suffix = ''.join(f'_{k}:{v[0]}{v[1]}' for (k, v) in sorted(guesses.items()))
	return f'bulls_n_cows_map/{digits}{suffix}.pkl'


def initialize(originals, digits):
    filepath = parse_bulls_n_cows_map_name(digits=digits)
    if os.path.isfile(filepath):
        return

    bulls_n_cows_map = {}
    for i, _ in enumerate(originals):
        bulls_n_cows_map[i] = {}
        for di in range(digits+1):
            for dj in range(di+1):
                bulls_n_cows_map[i][(dj, di-dj)] = set()

    for i, source in tqdm(enumerate(originals), total=len(originals)):
        bulls_n_cows_map[i][(digits, 0)].add(i)
        for j in range(i):
            target = originals[j]
            bulls_n_cows = calculate_bulls_cows(source, target)
            bulls_n_cows_map[i][bulls_n_cows].add(j)
            bulls_n_cows_map[j][bulls_n_cows].add(i)

    with open(filepath, 'wb') as f:
    	pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)


def convert_bulls_n_cows_map(originals, bulls_n_cows_map):
	return {originals[i]: {bc: set(originals[j] for j in bulls_n_cows_map[i][bc]) for bc in bulls_n_cows_map[i] if len(bulls_n_cows_map[i][bc]) > 0} for i in bulls_n_cows_map}


def read_bulls_n_cows_map(digits, curr_guesses={}):
	filepath = parse_bulls_n_cows_map_name(digits, curr_guesses)
	if not os.path.isfile(filepath):
		return None

	with open(filepath, 'rb') as f:
		bulls_n_cows_map = pickle.load(f)

	return bulls_n_cows_map


def update_bulls_n_cows_map(org_idx_map, guess, bulls_n_cows, digits, curr_guesses={}):
    next_guesses = curr_guesses.copy()
    next_guesses[guess] = bulls_n_cows

    filepath = parse_bulls_n_cows_map_name(digits=digits, guesses=next_guesses)
    if os.path.isfile(filepath):
        return read_bulls_n_cows_map(digits=digits, curr_guesses=next_guesses)

    bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=curr_guesses)

    guess_idx = org_idx_map[guess]	
    candidates = bulls_n_cows_map[guess_idx][bulls_n_cows]

    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            bulls_n_cows_map[src_idx][bc] = bulls_n_cows_map[src_idx][bc].intersection(candidates)

    with open(filepath, 'wb') as f:
        pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)
        
    return bulls_n_cows_map


def calc_candidates(bulls_n_cows_map):
    candidates = set()
    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            candidates = candidates.union(bulls_n_cows_map[src_idx][bc])

    return candidates


def calc_entropy(bulls_n_cows_map):
    entropy_map = {}
    for src_idx in bulls_n_cows_map:
        entropy_map[src_idx] = entropy([len(bulls_n_cows_map[src_idx][bc]) for bc in bulls_n_cows_map[src_idx]], base=2)

    return entropy_map


def try_guess(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, verbose=False):
    if verbose:
        bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=guesses)
        print('--', convert_bulls_n_cows_map(originals=originals, bulls_n_cows_map=bulls_n_cows_map))

    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)
    
    if verbose:
        print('++', convert_bulls_n_cows_map(originals=originals, bulls_n_cows_map=bulls_n_cows_map))

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)
    # entropy_map = calc_entropy(bulls_n_cows_map={k: bulls_n_cows_map[k] for k in bulls_n_cows_map if k in candidates})
    entropy_map = calc_entropy(bulls_n_cows_map=bulls_n_cows_map)
    best_guess = max(entropy_map, key=entropy_map.get)

    print(guesses, f": {safe_log2(len(candidates)):.2f} bits - {entropy_map[best_guess]:.2f} bits ('{originals[best_guess]}')", [originals[c] for c in candidates])

    return len(candidates), originals[best_guess], entropy_map[best_guess]

In [4]:
class BullsNCows:
    def __init__(self, digits=4):
        permute = permutations([i for i in range(10)], digits)
        self.originals = [tuple_to_string(p) for p in list(permute)]
        self.org_idx_map = {org: idx for idx, org in enumerate(self.originals)}
        self.digits = digits
        self.reset()

        initialize(originals=self.originals, digits=4)


    def reset(self):
        self.guesses = {}
        self.secret = random.choice(self.originals)
        self.summary = [{
            "guess": "",
            "guess_result": "",
            "guess_actual_entropy": np.NaN,
            "candidate_count": len(self.originals),
            "candidate_entropy": safe_log2(len(self.originals)),
            "best_guess": "",
            "best_guess_entropy": np.NaN,
        }]


    def next(self):
        if len(self.guesses) > 0:
            guess = self.summary[-1]['best_guess']
        else:
            guess = random.choice(self.originals)
 
        guess_result = calculate_bulls_cows(self.secret, guess)
        candidate_count, best_guess, best_guess_entropy = try_guess(originals=self.originals, org_idx_map=self.org_idx_map, guesses=self.guesses, guess=guess, bulls_n_cows=guess_result, digits=self.digits)

        self.summary.append({
            "guess": guess,
            "guess_result": guess_result,
            "guess_actual_entropy": self.summary[-1]['candidate_entropy']-safe_log2(candidate_count),
            "candidate_count": candidate_count,
            "candidate_entropy": safe_log2(candidate_count),
            "best_guess": best_guess,
            "best_guess_entropy": best_guess_entropy,
        })

        return self


    def play(self, n_iter=10):
        self.reset()
        print(self.secret)
        while self.summary[-1]['candidate_count'] > 1 and n_iter > 0:
            self.next()
            n_iter -= 1

        return pd.DataFrame.from_dict(self.summary)

In [5]:
game = BullsNCows(digits=4)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='1234', bulls_n_cows=(0, 2), digits=4, verbose=False)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='0145', bulls_n_cows=(0, 1), digits=4, verbose=False)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='2467', bulls_n_cows=(0, 1), digits=4, verbose=False)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='3852', bulls_n_cows=(1, 0), digits=4, verbose=False)

{'1234': (0, 2)} : 10.30 bits - 2.94 bits ('0145') ['0125', '0126', '0127', '0128', '0129', '8125', '8126', '8127', '8129', '8140', '0145', '0146', '0147', '0148', '0149', '0152', '0153', '8145', '8146', '8147', '8149', '8152', '0162', '0163', '8153', '8162', '8163', '0172', '0173', '8172', '8173', '0182', '0183', '8192', '8193', '0192', '0193', '8301', '8302', '8310', '0315', '0316', '0317', '0318', '0319', '8315', '8316', '0325', '0326', '0327', '0328', '0329', '8319', '8320', '0345', '0346', '0347', '0348', '0349', '0351', '0352', '8345', '8346', '8347', '8349', '8351', '0361', '0362', '8352', '8361', '8362', '0371', '0372', '8371', '8372', '0381', '0382', '8391', '8392', '8401', '0391', '0392', '8402', '8403', '8410', '0415', '0416', '0417', '0418', '0419', '8415', '8416', '0425', '0426', '0427', '0428', '0429', '8419', '8420', '8425', '8426', '8427', '8429', '0451', '0452', '0453', '8451', '8452', '8453', '0461', '0462', '0463', '8461', '8462', '8463', '0471', '0472', '0473', '847

(6, '0691', 2.584962500721156)

In [82]:
game = BullsNCows(digits=2)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2)

{'54': (1, 0)} : 4.00 bits - 1.54 bits ('04') ['74', '04', '84', '14', '24', '94', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64']
{'54': (1, 0), '12': (0, 1)} : 1.00 bits - 1.00 bits ('01') ['24', '51']
{'54': (1, 0), '12': (0, 1), '24': (2, 0)} : 0.00 bits - 0.00 bits ('01') ['24']


(1, '01', 0.0)

In [83]:
game = BullsNCows(digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='514', bulls_n_cows=(0, 1), digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='138', bulls_n_cows=(2, 0), digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='239', bulls_n_cows=(1, 0), digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='016', bulls_n_cows=(0, 1), digits=3)

{'514': (0, 1)} : 7.98 bits - 2.36 bits ('021') ['021', '721', '025', '725', '031', '731', '035', '735', '740', '042', '043', '742', '046', '047', '048', '049', '743', '052', '053', '746', '056', '057', '058', '059', '061', '750', '752', '753', '065', '756', '758', '759', '071', '761', '765', '781', '075', '785', '081', '791', '085', '795', '091', '801', '095', '805', '102', '103', '106', '107', '108', '109', '120', '123', '821', '126', '127', '128', '129', '130', '132', '825', '831', '136', '137', '138', '139', '835', '840', '842', '843', '846', '847', '849', '850', '852', '853', '856', '857', '859', '160', '162', '163', '861', '167', '168', '169', '170', '172', '173', '865', '871', '176', '178', '179', '180', '182', '183', '875', '891', '186', '187', '189', '190', '192', '193', '895', '901', '196', '197', '198', '201', '905', '205', '921', '925', '231', '931', '235', '935', '748', '240', '940', '243', '942', '246', '247', '248', '249', '250', '749', '253', '943', '256', '257', '258',

(1, '012', 0.0)

In [ ]:
df = BullsNCows(digits=4).play(); df

4932
{'1859': (0, 1)} : 10.49 bits - 2.86 bits ('0123') ['0123', '0124', '0126', '0127', '0132', '0134', '0136', '0137', '0142', '0143', '0146', '0147', '0162', '0163', '0164', '0167', '0172', '0173', '0174', '0176', '8203', '8204', '8206', '8207', '0213', '0214', '0216', '0217', '8230', '0231', '8234', '0235', '8236', '8237', '0238', '8240', '0241', '8243', '0245', '8246', '8247', '0248', '8260', '0261', '8263', '8264', '0265', '8267', '0268', '8270', '0271', '8273', '8274', '0275', '8276', '0278', '0283', '0284', '0286', '0287', '8302', '0293', '0294', '8304', '0296', '0297', '8306', '0312', '0314', '8307', '0316', '0317', '8320', '0321', '8324', '0325', '8326', '8327', '0328', '8340', '0341', '8342', '0345', '8346', '8347', '0348', '8360', '0361', '8362', '8364', '0365', '8367', '0368', '8370', '0371', '8372', '8374', '0375', '8376', '0378', '0382', '0384', '0386', '0387', '8402', '0392', '0394', '8403', '0396', '0397', '8406', '0412', '0413', '8407', '0416', '0417', '8420', '0421',

,guess,guess_result,guess_actual_entropy,candidate_count,candidate_entropy,best_guess,best_guess_entropy
0,,,NaN,5040,12.299208,,NaN
1,1859,"(0, 1)",1.807355,1440,10.491853,0123,2.858865
2,0123,"(0, 2)",1.964376,369,8.527477,8402,3.085136
3,8402,"(1, 1)",2.620586,60,5.906891,6308,3.261908
4,6308,"(0, 1)",2.321928,12,3.584963,4352,3.251629
5,4352,"(2, 1)",3.584963,1,0.000000,0123,0.000000
